In [ ]:
import numpy as np
from pynq import Overlay
import time

In [ ]:
ol = Overlay('svd.bit')

In [ ]:
ol?

In [ ]:
dma_main = ol.axi_dma_0
dma2 = ol.axi_dma_1
dma3 = ol.axi_dma_2

In [ ]:
def rmse(A, B):
    return np.sqrt(np.mean(np.abs(A - B) ** 2))


In [ ]:
dma_main?

In [ ]:
A_channel = dma_main.sendchannel
S_channel = dma_main.recvchannel
U_channel = dma2.recvchannel
V_channel = dma3.recvchannel

In [ ]:
import random

In [ ]:
np.random.seed(1)
data_A = (np.random.uniform(1, 2, (100, 100)) + 1j * np.random.uniform(1, 2, (100, 100))).astype(np.complex64)

In [ ]:
data_A

array([[1.417022 +1.765497j , 1.7203245+1.3081728j, 1.0001143+1.7353779j,
        ..., 1.5736794+1.4580276j, 1.0028703+1.7633603j,
        1.617145 +1.8640403j],
       [1.3266449+1.7079275j, 1.5270581+1.9480172j, 1.8859421+1.0469936j,
        ..., 1.2343621+1.0867507j, 1.6167784+1.4730929j,
        1.9490163+1.0767261j],
       [1.9501761+1.5781839j, 1.5566531+1.6910305j, 1.9156064+1.674659j ,
        ..., 1.0796261+1.7934036j, 1.9828172+1.9112906j,
        1.1816128+1.2699152j],
       ...,
       [1.2478703+1.505605j , 1.1118414+1.1462804j, 1.5135407+1.8063539j,
        ..., 1.8352761+1.3048706j, 1.2392855+1.5362678j,
        1.0730797+1.8590825j],
       [1.9529665+1.9167907j, 1.112327 +1.5760899j, 1.8023965+1.954928j ,
        ..., 1.4958543+1.8081243j, 1.5083709+1.4923786j,
        1.0847334+1.150792j ],
       [1.4372681+1.602121j , 1.8632013+1.8250479j, 1.6802363+1.2774987j,
        ..., 1.0595337+1.0451889j, 1.1080437+1.6965281j,
        1.7627938+1.2996428j]], dtype=complex64

In [ ]:
np.shape(data_A)

(100, 100)

In [ ]:
start = time.time()
u,s,v = np.linalg.svd(data_A)
fin = time.time()
nptime = fin-start
print(nptime)

0.050591230392456055


In [ ]:
s

array([2.1239891e+02, 7.9232202e+00, 7.7498188e+00, 7.4256849e+00,
       7.3009615e+00, 7.1953783e+00, 7.0889516e+00, 7.0073423e+00,
       6.8322587e+00, 6.7502604e+00, 6.5953059e+00, 6.4368720e+00,
       6.3563604e+00, 6.2376218e+00, 6.1396761e+00, 6.0253992e+00,
       5.9305186e+00, 5.7847404e+00, 5.7452512e+00, 5.6836548e+00,
       5.5901127e+00, 5.4566302e+00, 5.4110422e+00, 5.3150811e+00,
       5.1862350e+00, 5.1672440e+00, 5.1031013e+00, 5.0501833e+00,
       4.9133921e+00, 4.8738313e+00, 4.7843952e+00, 4.6514168e+00,
       4.6078901e+00, 4.4847879e+00, 4.4687352e+00, 4.3939147e+00,
       4.3053241e+00, 4.2505522e+00, 4.1781287e+00, 4.1098132e+00,
       4.0341911e+00, 3.9096129e+00, 3.8298752e+00, 3.7162254e+00,
       3.6954956e+00, 3.6592035e+00, 3.5320995e+00, 3.5025935e+00,
       3.4107082e+00, 3.3040106e+00, 3.2158689e+00, 3.1440918e+00,
       3.1039593e+00, 3.0684972e+00, 3.0071976e+00, 2.9210768e+00,
       2.8679597e+00, 2.8046882e+00, 2.7315369e+00, 2.6370990e

In [ ]:
np.shape(s)

(100,)

In [ ]:
np.shape(v)

(100, 100)

In [ ]:
from pynq import allocate

In [ ]:
input_buffer = allocate(100*100, np.complex64)
out1_buff = allocate(100*100, np.complex64)
out2_buff = allocate(100*100,np.complex64)
out3_buff = allocate(100*100,np.complex64)

In [ ]:
np.copyto(input_buffer, data_A.flatten())

In [ ]:
V_channel

In [ ]:
U_channel

In [ ]:
start_time = time.time()
V_channel.transfer(out3_buff)
U_channel.transfer(out2_buff)
A_channel.transfer(input_buffer)
S_channel.transfer(out1_buff)
A_channel.wait()
S_channel.wait()
V_channel.wait()
U_channel.wait()
end_time = time.time()
pltime = end_time - start_time
print(pltime)

1.5750408172607422


In [ ]:
out1_buff

PynqBuffer([ 3.7725028e-02+0.0000000e+00j,  1.2197466e-09-3.6348178e-09j,
             0.0000000e+00+0.0000000e+00j, ...,
             0.0000000e+00+0.0000000e+00j, -3.1253685e-28+9.2898974e-28j,
             2.1239731e+02+0.0000000e+00j], dtype=complex64)

In [ ]:
out2_buff

PynqBuffer([ 0.00585971+0.05686354j,  0.07608929+0.12933616j,
             0.00149604+0.00125614j, ..., -0.10884618-0.04859803j,
            -0.03277247-0.09807242j, -0.07962636-0.06167038j],
           dtype=complex64)

In [ ]:
out3_buff

PynqBuffer([ 0.0091247 -0.04821643j,  0.04709733-0.10622156j,
            -0.0873123 +0.12684083j, ...,  0.00094319-0.04306549j,
             0.01687942-0.00564517j, -0.10209162+0.01285029j],
           dtype=complex64)

In [ ]:
u,s,v = np.linalg.svd(data_A)

In [ ]:
s

array([2.1239891e+02, 7.9232202e+00, 7.7498188e+00, 7.4256849e+00,
       7.3009615e+00, 7.1953783e+00, 7.0889516e+00, 7.0073423e+00,
       6.8322587e+00, 6.7502604e+00, 6.5953059e+00, 6.4368720e+00,
       6.3563604e+00, 6.2376218e+00, 6.1396761e+00, 6.0253992e+00,
       5.9305186e+00, 5.7847404e+00, 5.7452512e+00, 5.6836548e+00,
       5.5901127e+00, 5.4566302e+00, 5.4110422e+00, 5.3150811e+00,
       5.1862350e+00, 5.1672440e+00, 5.1031013e+00, 5.0501833e+00,
       4.9133921e+00, 4.8738313e+00, 4.7843952e+00, 4.6514168e+00,
       4.6078901e+00, 4.4847879e+00, 4.4687352e+00, 4.3939147e+00,
       4.3053241e+00, 4.2505522e+00, 4.1781287e+00, 4.1098132e+00,
       4.0341911e+00, 3.9096129e+00, 3.8298752e+00, 3.7162254e+00,
       3.6954956e+00, 3.6592035e+00, 3.5320995e+00, 3.5025935e+00,
       3.4107082e+00, 3.3040106e+00, 3.2158689e+00, 3.1440918e+00,
       3.1039593e+00, 3.0684972e+00, 3.0071976e+00, 2.9210768e+00,
       2.8679597e+00, 2.8046882e+00, 2.7315369e+00, 2.6370990e

In [ ]:
out1_buff

PynqBuffer([ 3.7725028e-02+0.0000000e+00j,  1.2197466e-09-3.6348178e-09j,
             0.0000000e+00+0.0000000e+00j, ...,
             0.0000000e+00+0.0000000e+00j, -3.1253685e-28+9.2898974e-28j,
             2.1239731e+02+0.0000000e+00j], dtype=complex64)

In [ ]:
s2 = np.sort(s)

In [ ]:
sigma = np.diag(s2)

In [ ]:
rmse(out1_buff,sigma.flatten())

PynqBuffer(1.6452332e-05, dtype=float32)

In [ ]:
rmse(out2_buff, u.flatten())

PynqBuffer(0.1414209, dtype=float32)

In [ ]:
u2_pl = np.sort(np.abs(out2_buff))

In [ ]:
u2_np = np.sort(np.abs(u.flatten()))

In [ ]:
rmse(u2_pl,u2_np)

PynqBuffer(6.010581e-06, dtype=float32)

In [ ]:
out2_buff

PynqBuffer([ 0.00585971+0.05686354j,  0.07608929+0.12933616j,
             0.00149604+0.00125614j, ..., -0.10884618-0.04859803j,
            -0.03277247-0.09807242j, -0.07962636-0.06167038j],
           dtype=complex64)

In [ ]:
u.flatten()

array([-0.07110938-0.07152744j, -0.04677915-0.00940512j,
       -0.11675371-0.04313587j, ..., -0.00190734-0.01351257j,
        0.02151629-0.02604547j,  0.15266314-0.03508801j], dtype=complex64)

In [ ]:
v.flatten()

array([-0.09936834+0.j        , -0.10092956-0.00091883j,
       -0.09968395-0.0027048j , ..., -0.03110069-0.00496953j,
       -0.09578112-0.02714771j, -0.05650753-0.03951763j], dtype=complex64)

In [ ]:
out3_buff

PynqBuffer([ 0.0091247 -0.04821643j,  0.04709733-0.10622156j,
            -0.0873123 +0.12684083j, ...,  0.00094319-0.04306549j,
             0.01687942-0.00564517j, -0.10209162+0.01285029j],
           dtype=complex64)

In [ ]:
v2_pl = np.sort(np.abs(out3_buff))

In [ ]:
v2_np = np.sort(np.abs(u.flatten()))

In [ ]:
rmse(v2_pl,v2_np)

PynqBuffer(0.00078252, dtype=float32)

In [ ]:
v.flatten()

array([-0.09936834+0.j        , -0.10092956-0.00091883j,
       -0.09968395-0.0027048j , ..., -0.03110069-0.00496953j,
       -0.09578112-0.02714771j, -0.05650753-0.03951763j], dtype=complex64)

In [ ]:
out3_buff

PynqBuffer([ 0.0091247 -0.04821643j,  0.04709733-0.10622156j,
            -0.0873123 +0.12684083j, ...,  0.00094319-0.04306549j,
             0.01687942-0.00564517j, -0.10209162+0.01285029j],
           dtype=complex64)

In [ ]:
u.flatten()

array([-0.07110938-0.07152744j, -0.04677915-0.00940512j,
       -0.11675371-0.04313587j, ..., -0.00190734-0.01351257j,
        0.02151629-0.02604547j,  0.15266314-0.03508801j], dtype=complex64)

In [ ]:
out2_buff

PynqBuffer([ 0.00585971+0.05686354j,  0.07608929+0.12933616j,
             0.00149604+0.00125614j, ..., -0.10884618-0.04859803j,
            -0.03277247-0.09807242j, -0.07962636-0.06167038j],
           dtype=complex64)

In [ ]:
sigma_pl = np.reshape(out1_buff, (100,100))
u_pl = np.reshape(out2_buff, (100,100))
v_pl = np.reshape(out3_buff, (100,100))

In [ ]:
u_pl @ sigma_pl @v_pl.T

PynqBuffer([[1.3696663+1.1095454j , 1.6128511+1.7995255j ,
             1.748347 +1.3539358j , ..., 1.1538718+1.5595678j ,
             3.0150704+1.2392017j , 2.2309864+1.4570053j ],
            [1.3974541+1.2684461j , 1.6615422+0.96626353j,
             1.6576121+0.06731851j, ..., 1.7749643+1.0100819j ,
             2.0056798+0.90999883j, 1.5945357+0.85978323j],
            [1.712533 +1.3821484j , 2.188373 +1.2710946j ,
             1.6277766+0.7363799j , ..., 1.9151777+1.1618193j ,
             1.9751362+0.9805975j , 2.0522213+0.87826306j],
            ...,
            [1.7361352+1.3384023j , 2.324238 +1.2573555j ,
             1.8973131+0.8507204j , ..., 2.0317671+1.2989159j ,
             1.64364  +0.8832249j , 2.156201 +1.0889366j ],
            [1.4051172+0.84777296j, 1.566442 +0.90892065j,
             2.2138567+0.9033984j , ..., 1.5117961+1.2145948j ,
             1.59773  +0.6935471j , 2.6087003+1.5117348j ],
            [1.660774 +0.60415214j, 1.7817367+1.3588856j ,
         

In [ ]:
data_A

array([[1.417022 +1.765497j , 1.7203245+1.3081728j, 1.0001143+1.7353779j,
        ..., 1.5736794+1.4580276j, 1.0028703+1.7633603j,
        1.617145 +1.8640403j],
       [1.3266449+1.7079275j, 1.5270581+1.9480172j, 1.8859421+1.0469936j,
        ..., 1.2343621+1.0867507j, 1.6167784+1.4730929j,
        1.9490163+1.0767261j],
       [1.9501761+1.5781839j, 1.5566531+1.6910305j, 1.9156064+1.674659j ,
        ..., 1.0796261+1.7934036j, 1.9828172+1.9112906j,
        1.1816128+1.2699152j],
       ...,
       [1.2478703+1.505605j , 1.1118414+1.1462804j, 1.5135407+1.8063539j,
        ..., 1.8352761+1.3048706j, 1.2392855+1.5362678j,
        1.0730797+1.8590825j],
       [1.9529665+1.9167907j, 1.112327 +1.5760899j, 1.8023965+1.954928j ,
        ..., 1.4958543+1.8081243j, 1.5083709+1.4923786j,
        1.0847334+1.150792j ],
       [1.4372681+1.602121j , 1.8632013+1.8250479j, 1.6802363+1.2774987j,
        ..., 1.0595337+1.0451889j, 1.1080437+1.6965281j,
        1.7627938+1.2996428j]], dtype=complex64

In [ ]:
rmse(u_pl @ sigma_pl @v_pl.T, data_A)

PynqBuffer(0.7816028, dtype=float32)

In [ ]:
v.T

array([[-0.09936834+0.j        , -0.14909948+0.j        ,
         0.17442486+0.j        , ..., -0.15398961+0.j        ,
         0.11619286+0.j        ,  0.04907367+0.j        ],
       [-0.10092956-0.00091883j,  0.06196608-0.03076945j,
        -0.001803  -0.00462422j, ...,  0.00306896+0.01672518j,
         0.04622827+0.12920268j, -0.00946623-0.02566963j],
       [-0.09968395-0.0027048j ,  0.04155092-0.03416482j,
        -0.03160503-0.09078731j, ...,  0.00146419+0.0372038j ,
         0.07536043-0.05333554j, -0.06368874-0.0387369j ],
       ...,
       [-0.10023212-0.0018346j , -0.04191505-0.04982j   ,
        -0.03786309-0.06721666j, ..., -0.16885784-0.05892014j,
        -0.04221101-0.04857469j, -0.03110069-0.00496953j],
       [-0.1008413 -0.00072555j,  0.01719541+0.0114174j ,
         0.04692801-0.13566303j, ..., -0.00491535-0.01232928j,
        -0.06351992-0.05405173j, -0.09578112-0.02714771j],
       [-0.1028825 -0.00180098j, -0.00287444-0.01756347j,
        -0.03153005+0.02934995

In [ ]:
v_pl

PynqBuffer([[ 0.0091247 -0.04821643j,  0.04709733-0.10622156j,
             -0.0873123 +0.12684083j, ...,  0.11600776+0.13023679j,
              0.0695036 +0.1319172j , -0.0987914 +0.01068164j],
            [ 0.02346168+0.01407334j, -0.09937623-0.09462921j,
             -0.01203832-0.01201196j, ..., -0.00465155+0.00173172j,
             -0.00166238-0.06916403j, -0.10024444+0.01176354j],
            [ 0.0262183 +0.06978102j,  0.07930419-0.04727371j,
             -0.0298135 -0.02230615j, ..., -0.0888118 +0.03678966j,
              0.01085211-0.05268427j, -0.0988147 +0.01340507j],
            ...,
            [-0.00090131+0.03148311j,  0.0273005 +0.05827395j,
             -0.04720167+0.17249386j, ..., -0.07537923+0.0164342j ,
              0.06360866+0.01386177j, -0.09945288+0.01259874j],
            [ 0.00886407+0.09915724j,  0.02366663+0.07997573j,
              0.00736796+0.01103635j, ..., -0.0700863 +0.12527713j,
             -0.01811996-0.00988602j, -0.10017782+0.0115616j ],
        

In [ ]:
np.max(u2_pl)

PynqBuffer(0.29156774, dtype=float32)

In [ ]:
np.min(u2_np)

0.0015242893

In [ ]:
np.max(v2_pl)

PynqBuffer(0.29463497, dtype=float32)

In [ ]:

np.min(v2_np)

0.0015242893

In [ ]:
np.max(sigma)

212.39891

In [ ]:
np.min(s)

0.037725497

In [ ]:
out1_buff

PynqBuffer([ 3.7725028e-02+0.0000000e+00j,  1.2197466e-09-3.6348178e-09j,
             0.0000000e+00+0.0000000e+00j, ...,
             0.0000000e+00+0.0000000e+00j, -3.1253685e-28+9.2898974e-28j,
             2.1239731e+02+0.0000000e+00j], dtype=complex64)

In [ ]:
s

array([2.1239891e+02, 7.9232202e+00, 7.7498188e+00, 7.4256849e+00,
       7.3009615e+00, 7.1953783e+00, 7.0889516e+00, 7.0073423e+00,
       6.8322587e+00, 6.7502604e+00, 6.5953059e+00, 6.4368720e+00,
       6.3563604e+00, 6.2376218e+00, 6.1396761e+00, 6.0253992e+00,
       5.9305186e+00, 5.7847404e+00, 5.7452512e+00, 5.6836548e+00,
       5.5901127e+00, 5.4566302e+00, 5.4110422e+00, 5.3150811e+00,
       5.1862350e+00, 5.1672440e+00, 5.1031013e+00, 5.0501833e+00,
       4.9133921e+00, 4.8738313e+00, 4.7843952e+00, 4.6514168e+00,
       4.6078901e+00, 4.4847879e+00, 4.4687352e+00, 4.3939147e+00,
       4.3053241e+00, 4.2505522e+00, 4.1781287e+00, 4.1098132e+00,
       4.0341911e+00, 3.9096129e+00, 3.8298752e+00, 3.7162254e+00,
       3.6954956e+00, 3.6592035e+00, 3.5320995e+00, 3.5025935e+00,
       3.4107082e+00, 3.3040106e+00, 3.2158689e+00, 3.1440918e+00,
       3.1039593e+00, 3.0684972e+00, 3.0071976e+00, 2.9210768e+00,
       2.8679597e+00, 2.8046882e+00, 2.7315369e+00, 2.6370990e

In [ ]:
c = np.reshape(out2_buff, (100,100))

In [ ]:
u

array([[-0.07110938-0.07152744j, -0.04677915-0.00940512j,
        -0.11675371-0.04313587j, ..., -0.00018357+0.00193972j,
        -0.08739489+0.12198317j, -0.05478054+0.01633248j],
       [-0.07104395-0.06896158j, -0.0692672 +0.09445859j,
         0.05901126+0.0152944j , ...,  0.04362869+0.06780671j,
        -0.05288525-0.09455415j, -0.054092  +0.03217037j],
       [-0.07432389-0.07074448j,  0.01591112-0.05393297j,
         0.00291301+0.02607743j, ...,  0.02442739-0.09558404j,
         0.05246083+0.02013403j, -0.0422301 -0.05773035j],
       ...,
       [-0.07119677-0.06890884j, -0.04460623-0.01561402j,
         0.04775504-0.05980061j, ...,  0.11201281-0.09145534j,
         0.08721732-0.11910616j, -0.03896205-0.0719882j ],
       [-0.07527205-0.06764916j, -0.05816168-0.10160465j,
         0.09819489+0.06595337j, ...,  0.01339313+0.08762023j,
        -0.08270279-0.14756997j,  0.05838583+0.12185659j],
       [-0.07253632-0.06987338j,  0.10203773+0.01672416j,
        -0.10869282+0.04895189

In [ ]:
c

PynqBuffer([[ 0.00585971+0.05686354j,  0.07608929+0.12933616j,
              0.00149604+0.00125614j, ..., -0.04544783-0.11587597j,
              0.0134838 +0.04576318j, -0.07838526-0.06346739j],
            [ 0.02155141+0.05913151j, -0.10787366+0.01002238j,
              0.08058834+0.00251594j, ...,  0.02782498+0.0542385j ,
              0.11585845+0.01725438j, -0.0780448 -0.06092444j],
            [-0.06457638+0.03075837j,  0.03966837-0.03979406j,
             -0.06488558-0.07431824j, ..., -0.01753213+0.01951984j,
             -0.05513161+0.01106444j, -0.0814974 -0.06234439j],
            ...,
            [-0.07797647+0.02489281j, -0.07353292-0.12800571j,
             -0.01182876-0.1441199j , ...,  0.0764186 -0.00411868j,
              0.00698211+0.04674081j, -0.07819077-0.06085543j],
            [ 0.13058645-0.03470654j, -0.16842705+0.01578865j,
              0.07976355+0.03865094j, ...,  0.01606682+0.11718532j,
             -0.06278126+0.09882316j, -0.08210725-0.05916498j],
        

In [ ]:
d = np.reshape(out3_buff, (100,100))

In [ ]:
v.T

array([[-0.09936834+0.j        , -0.14909948+0.j        ,
         0.17442486+0.j        , ..., -0.15398961+0.j        ,
         0.11619286+0.j        ,  0.04907367+0.j        ],
       [-0.10092956-0.00091883j,  0.06196608-0.03076945j,
        -0.001803  -0.00462422j, ...,  0.00306896+0.01672518j,
         0.04622827+0.12920268j, -0.00946623-0.02566963j],
       [-0.09968395-0.0027048j ,  0.04155092-0.03416482j,
        -0.03160503-0.09078731j, ...,  0.00146419+0.0372038j ,
         0.07536043-0.05333554j, -0.06368874-0.0387369j ],
       ...,
       [-0.10023212-0.0018346j , -0.04191505-0.04982j   ,
        -0.03786309-0.06721666j, ..., -0.16885784-0.05892014j,
        -0.04221101-0.04857469j, -0.03110069-0.00496953j],
       [-0.1008413 -0.00072555j,  0.01719541+0.0114174j ,
         0.04692801-0.13566303j, ..., -0.00491535-0.01232928j,
        -0.06351992-0.05405173j, -0.09578112-0.02714771j],
       [-0.1028825 -0.00180098j, -0.00287444-0.01756347j,
        -0.03153005+0.02934995

In [ ]:
d

PynqBuffer([[ 0.0091247 -0.04821643j,  0.04709733-0.10622156j,
             -0.0873123 +0.12684083j, ...,  0.11600776+0.13023679j,
              0.0695036 +0.1319172j , -0.0987914 +0.01068164j],
            [ 0.02346168+0.01407334j, -0.09937623-0.09462921j,
             -0.01203832-0.01201196j, ..., -0.00465155+0.00173172j,
             -0.00166238-0.06916403j, -0.10024444+0.01176354j],
            [ 0.0262183 +0.06978102j,  0.07930419-0.04727371j,
             -0.0298135 -0.02230615j, ..., -0.0888118 +0.03678966j,
              0.01085211-0.05268427j, -0.0988147 +0.01340507j],
            ...,
            [-0.00090131+0.03148311j,  0.0273005 +0.05827395j,
             -0.04720167+0.17249386j, ..., -0.07537923+0.0164342j ,
              0.06360866+0.01386177j, -0.09945288+0.01259874j],
            [ 0.00886407+0.09915724j,  0.02366663+0.07997573j,
              0.00736796+0.01103635j, ..., -0.0700863 +0.12527713j,
             -0.01811996-0.00988602j, -0.10017782+0.0115616j ],
        

In [ ]:
np.min(data_A)

(1.0000969+1.6635365j)

In [ ]:
np.max(data_A)

(1.9998758+1.8338038j)

In [ ]:
u.flatten()

array([-0.07110938-0.07152744j, -0.04677915-0.00940512j,
       -0.11675371-0.04313587j, ..., -0.00190734-0.01351257j,
        0.02151629-0.02604547j,  0.15266314-0.03508801j], dtype=complex64)

In [ ]:
out2_buff

PynqBuffer([ 0.00585971+0.05686354j,  0.07608929+0.12933616j,
             0.00149604+0.00125614j, ..., -0.10884618-0.04859803j,
            -0.03277247-0.09807242j, -0.07962636-0.06167038j],
           dtype=complex64)

In [ ]:
f = np.sort(u.flatten())

In [ ]:
g = np.sort(out2_buff)

In [ ]:
f

array([-0.24811976+0.05564642j, -0.24072589-0.03868587j,
       -0.23891512+0.06196057j, ...,  0.2291312 -0.11531449j,
        0.23830716-0.01478987j,  0.26847556+0.06080777j], dtype=complex64)

In [ ]:
g

PynqBuffer([-0.2553671 +0.02416432j, -0.22764328+0.057572j  ,
            -0.22392027+0.07505751j, ...,  0.23651648+0.02987482j,
             0.2546313 +0.02944133j,  0.26927933+0.09785754j],
           dtype=complex64)

In [ ]:
rmse(f,g)

PynqBuffer(0.09982529, dtype=float32)

In [ ]:
np.min(data_A)

(1.0000969+1.6635365j)

In [ ]:
np.max(data_A)

(1.9998758+1.8338038j)

In [ ]:
def pinv_from_svd(U, Sigma, Vh):
    tol=1e-10
    m, n = Sigma.shape
    sigma_vals = Sigma.diagonal()[:min(Sigma.shape)]
    sigma_inv = np.array([1/s if np.abs(s) > tol else 0 for s in sigma_vals])
    Sigma_inv = np.zeros((n, m), dtype=complex)
    for i in range(len(sigma_inv)):
        Sigma_inv[i, i] = sigma_inv[i]
    return Vh.conj().T @ Sigma_inv @ U.conj().T


In [ ]:
U = np.zeros(100*100).astype(np.complex64)
S = np.zeros(100*100).astype(np.complex64)
V = np.zeros(100*100).astype(np.complex64)

In [ ]:
np.copyto(U, out2_buff)
np.copyto(S, out1_buff)
np.copyto(V, out3_buff)

In [ ]:
U

array([ 0.00585971+0.05686354j,  0.07608929+0.12933616j,
        0.00149604+0.00125614j, ..., -0.10884618-0.04859803j,
       -0.03277247-0.09807242j, -0.07962636-0.06167038j], dtype=complex64)

In [ ]:
S

array([ 3.7725028e-02+0.0000000e+00j,  1.2197466e-09-3.6348178e-09j,
        0.0000000e+00+0.0000000e+00j, ...,  0.0000000e+00+0.0000000e+00j,
       -3.1253685e-28+9.2898974e-28j,  2.1239731e+02+0.0000000e+00j],
      dtype=complex64)

In [ ]:
V

array([ 0.0091247 -0.04821643j,  0.04709733-0.10622156j,
       -0.0873123 +0.12684083j, ...,  0.00094319-0.04306549j,
        0.01687942-0.00564517j, -0.10209162+0.01285029j], dtype=complex64)

In [ ]:
U = np.reshape(U, (100,100))

In [ ]:
V = np.reshape(V, (100,100))

In [ ]:
S = np.reshape(S, (100,100))

In [ ]:
U@S@V.conj().T

array([[1.4169784+1.7654263j, 1.7202613+1.3081114j, 1.0000792+1.735317j ,
        ..., 1.5736271+1.4579657j, 1.0028305+1.7632921j,
        1.617089 +1.8639727j],
       [1.3266088+1.7078782j, 1.527005 +1.947968j , 1.8858908+1.0469723j,
        ..., 1.234323 +1.0867257j, 1.6167322+1.4730501j,
        1.9489602+1.0766976j],
       [1.9501266+1.5781404j, 1.5565944+1.6909903j, 1.9155608+1.6746268j,
        ..., 1.0795971+1.7933598j, 1.9827592+1.9112432j,
        1.181572 +1.2698835j],
       ...,
       [1.2478333+1.5055559j, 1.1117915+1.1462448j, 1.5134946+1.8063086j,
        ..., 1.8352144+1.304833j , 1.2392404+1.5362264j,
        1.0730375+1.8590351j],
       [1.9529133+1.9167329j, 1.1122816+1.576044j , 1.8023489+1.9548861j,
        ..., 1.4958074+1.8080751j, 1.5083243+1.4923396j,
        1.0847002+1.1507604j],
       [1.4372278+1.602074j , 1.8631314+1.8250028j, 1.6801852+1.2774726j,
        ..., 1.0594984+1.0451622j, 1.1080054+1.6964834j,
        1.762739 +1.2996082j]], dtype=complex64

In [ ]:
data_A

array([[1.417022 +1.765497j , 1.7203245+1.3081728j, 1.0001143+1.7353779j,
        ..., 1.5736794+1.4580276j, 1.0028703+1.7633603j,
        1.617145 +1.8640403j],
       [1.3266449+1.7079275j, 1.5270581+1.9480172j, 1.8859421+1.0469936j,
        ..., 1.2343621+1.0867507j, 1.6167784+1.4730929j,
        1.9490163+1.0767261j],
       [1.9501761+1.5781839j, 1.5566531+1.6910305j, 1.9156064+1.674659j ,
        ..., 1.0796261+1.7934036j, 1.9828172+1.9112906j,
        1.1816128+1.2699152j],
       ...,
       [1.2478703+1.505605j , 1.1118414+1.1462804j, 1.5135407+1.8063539j,
        ..., 1.8352761+1.3048706j, 1.2392855+1.5362678j,
        1.0730797+1.8590825j],
       [1.9529665+1.9167907j, 1.112327 +1.5760899j, 1.8023965+1.954928j ,
        ..., 1.4958543+1.8081243j, 1.5083709+1.4923786j,
        1.0847334+1.150792j ],
       [1.4372681+1.602121j , 1.8632013+1.8250479j, 1.6802363+1.2774987j,
        ..., 1.0595337+1.0451889j, 1.1080437+1.6965281j,
        1.7627938+1.2996428j]], dtype=complex64

In [ ]:
rmse(U@S@V.conj().T, data_A)

6.559488e-05

In [ ]:
Sigma = S

In [ ]:
tol = 1e-10
m, n = Sigma.shape
Sigma_inv = np.zeros((n, m), dtype=complex)

for i in range(min(m, n)):
    if abs(Sigma[i, i]) > tol:
        Sigma_inv[i, i] = 1 / Sigma[i, i]


In [ ]:
PL_PINV = V @ Sigma_inv @ U.conj().T

In [ ]:
PS_PINV = np.linalg.pinv(data_A)

In [ ]:
rmse(PS_PINV, PL_PINV)

6.503388016212541e-06

In [ ]:
s2

array([-2.5279433e-06-2.7564840e-06j, -1.7423891e-06+5.7909278e-07j,
       -1.4182103e-06-1.3981178e-06j, ...,  7.7497406e+00+0.0000000e+00j,
        7.9231472e+00+0.0000000e+00j,  2.1239731e+02+0.0000000e+00j],
      dtype=complex64)

In [ ]:
s

array([2.1239891e+02, 7.9232202e+00, 7.7498188e+00, 7.4256849e+00,
       7.3009615e+00, 7.1953783e+00, 7.0889516e+00, 7.0073423e+00,
       6.8322587e+00, 6.7502604e+00, 6.5953059e+00, 6.4368720e+00,
       6.3563604e+00, 6.2376218e+00, 6.1396761e+00, 6.0253992e+00,
       5.9305186e+00, 5.7847404e+00, 5.7452512e+00, 5.6836548e+00,
       5.5901127e+00, 5.4566302e+00, 5.4110422e+00, 5.3150811e+00,
       5.1862350e+00, 5.1672440e+00, 5.1031013e+00, 5.0501833e+00,
       4.9133921e+00, 4.8738313e+00, 4.7843952e+00, 4.6514168e+00,
       4.6078901e+00, 4.4847879e+00, 4.4687352e+00, 4.3939147e+00,
       4.3053241e+00, 4.2505522e+00, 4.1781287e+00, 4.1098132e+00,
       4.0341911e+00, 3.9096129e+00, 3.8298752e+00, 3.7162254e+00,
       3.6954956e+00, 3.6592035e+00, 3.5320995e+00, 3.5025935e+00,
       3.4107082e+00, 3.3040106e+00, 3.2158689e+00, 3.1440918e+00,
       3.1039593e+00, 3.0684972e+00, 3.0071976e+00, 2.9210768e+00,
       2.8679597e+00, 2.8046882e+00, 2.7315369e+00, 2.6370990e

In [ ]:
s = np.diag(s).flatten()
s = np.sort(s)

In [ ]:
rmse(s,s2)

1.6452332e-05

In [ ]:
np.shape(U)

(100, 100)

In [ ]:
X = U.flatten()

In [ ]:
np.shape(X)

(10000,)

In [ ]:
u2.flatten()

array([-0.2553671 +0.02416432j, -0.22764328+0.057572j  ,
       -0.22392027+0.07505751j, ...,  0.23651648+0.02987482j,
        0.2546313 +0.02944133j,  0.26927933+0.09785754j], dtype=complex64)

In [ ]:
rmse(u,u2)

0.099825285

In [ ]:
rmse(v,v2)

0.100330286